In [1]:
import numpy as np

np.random.seed(12345)


def initialize(input_dim, hidden_dim, output_dim, batch_size):
    W1 = np.random.randn(hidden_dim, input_dim) * 0.01
    b1 = np.zeros((hidden_dim,))
    W2 = np.random.randn(output_dim, hidden_dim) * 0.01
    b2 = np.zeros((output_dim,))

    parameters = [W1, b1, W2, b2]
    x = np.random.rand(input_dim, batch_size)
    y = np.random.randn(output_dim, batch_size)

    return parameters, x, y


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def deriv_sigmoid(x):
    return x * (1 - x)

In [2]:
def forward(parameters, X):
    W1, b1, W2, b2 = parameters

    batch_size = X.shape[1]
    hidden_dim = W1.shape[0]
    output_dim = W2.shape[0]

    hid = np.zeros((hidden_dim, batch_size))
    outputs = np.zeros((output_dim, batch_size))

    hid = sigmoid(np.dot(W1, X) + b1.reshape(-1,1))
    outputs = np.dot(W2, hid) + b2.reshape(-1,1)

    activations = [X, hid, outputs]

    return activations

In [3]:
def squared_loss(predictions, targets):
    """ Computes mean squared error

    predictions: (output_dim, batch_size)
    targets: (output_dim, batch_size)

    """

    loss = np.zeros(targets.shape[1])

    loss = (1./targets.shape[1]) * np.sum(np.sum(.5 * (predictions - targets)**2, axis=0))

    return np.mean(loss)

In [4]:
def deriv_squared_loss(predictions, targets):
    
    batch_size = targets.shape[1]
    dloss = np.zeros(targets.shape)

    dloss = (predictions - targets) / batch_size

    return dloss

In [5]:
def backward(activations, targets, parameters):

    X, hid, predictions = activations

    input_dim = X.shape[0]
    hidden_dim = hid.shape[0]
    output_dim = predictions.shape[0]

    W1, b1, W2, b2 = parameters

    dW1 = np.zeros((hidden_dim, input_dim))
    db1 = np.zeros((hidden_dim,))
    dW2 = np.zeros((output_dim, hidden_dim))
    db2 = np.zeros((output_dim,))

    out_error = squared_loss(predictions, targets)
    out_delta = deriv_squared_loss(predictions, targets)
    
    dhid_error = np.dot(W2.T, out_delta)
    dhid_delta = dhid_error * deriv_sigmoid(hid)
    
    dW1 = np.dot(dhid_delta, X.T)
    db1 = np.sum(dhid_delta, axis=1)
    
    dW2 = np.dot(out_delta, hid.T)
    db2 = np.sum(out_delta, axis=1)

    grads = [dW1, db1, dW2, db2]

    return grads


In [6]:
def training(X_train, y_train, learning_rate):
    input_dim = X_train.shape[1]
    hidden_dim = 30
    output_dim = y_train.shape[1]
    batch_size = 5
    iteration = int(X_train.shape[0] / batch_size)
    
    parameters, X, Y = initialize(input_dim, hidden_dim, output_dim, batch_size)
    W1, b1, W2, b2 = parameters
    
    for i in range(0, iteration):
        
        X = X_train[i*batch_size:(i+1)*batch_size].T
        Y = y_train[i*batch_size:(i+1)*batch_size].T
        
        activations = forward(parameters, X)
        
        P = activations[-1]
        
        loss = squared_loss(P, Y)
        dW1, db1, dW2, db2 = backward(activations, Y, parameters)
        
        W1 += -dW1 * learning_rate
        b1 += -db1 * learning_rate
        W2 += -dW2 * learning_rate
        b2 += -db2 * learning_rate
        
    parameters = [W1, b1, W2, b2]
    
    return parameters      

In [7]:
def predict (parameters, X_test):
    X, hid, outputs = forward(parameters, X_test.T)
    y_predic = np.argmax(outputs, axis=0)
    return y_predic

In [8]:
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.model_selection import train_test_split

from sklearn.datasets import fetch_mldata

from sklearn.preprocessing import StandardScaler

from sklearn.utils import check_random_state

from sklearn.preprocessing import OneHotEncoder 
 

mnist = fetch_mldata('MNIST original')

X = mnist.data.astype('float64')

y = mnist.target

random_state = check_random_state(0)

 

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=1000, test_size=300, random_state= random_state)

##convert numerical label into one-hot-encoding
enc = OneHotEncoder()
enc.fit(y_train.reshape(-1, 1))  
y_train_one = enc.transform(y_train.reshape(-1, 1)).toarray()

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)


parameters = training(X_train, y_train_one, 0.1)
y_predic = predict (parameters, X_test)
 

expected = y_test

predicted = y_predic #Network output  

In [9]:
print("Classification report:")

print(classification_report(expected, predicted))

print("Confusion matrix:")

print(confusion_matrix(expected, predicted)) 

Classification report:
             precision    recall  f1-score   support

        0.0       1.00      0.84      0.92        32
        1.0       0.76      0.94      0.84        34
        2.0       0.58      0.91      0.71        32
        3.0       0.86      0.58      0.69        31
        4.0       0.51      0.96      0.67        26
        5.0       0.50      0.79      0.61        24
        6.0       0.93      0.96      0.95        27
        7.0       0.72      0.90      0.80        31
        8.0       0.83      0.16      0.27        31
        9.0       0.00      0.00      0.00        32

avg / total       0.67      0.70      0.64       300

Confusion matrix:
[[27  0  2  0  0  3  0  0  0  0]
 [ 0 32  1  0  0  1  0  0  0  0]
 [ 0  0 29  0  1  0  1  0  1  0]
 [ 0  0  8 18  0  4  0  1  0  0]
 [ 0  0  1  0 25  0  0  0  0  0]
 [ 0  1  2  0  1 19  1  0  0  0]
 [ 0  0  1  0  0  0 26  0  0  0]
 [ 0  2  1  0  0  0  0 28  0  0]
 [ 0  6  5  2  2 11  0  0  5  0]
 [ 0  1  0  1 20  0  0 

/home/zhiyuan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
